# Solving the Darcy-Flow problem using FNO

In this notebook, we will introduce the brief theory behind the Fourier Neural Operators and use them to solve a data-driven Darcy flow problem. The example is adapted from the [paper](https://arxiv.org/pdf/2010.08895.pdf) by Zongyi Li et al. You can also refer to the FNO example and theory from [PhysicsNeMo User Documentation](https://docs.nvidia.com/deeplearning/physicsnemo/physicsnemo-sym/user_guide/neural_operators/darcy_fno.html) for additional details. 


#### Contents of the Notebook

- [Introduction to Data-driven approach](#Introduction-to-Data-driven-approach)
- [Theory of the Fourier Neural Operator](#Theory-of-the-Fourier-Neural-Operator)
- [Solving the Darcy-Flow problem](#Solving-the-Darcy-Flow-problem)
    - [Problem Description](#Problem-Description)
    - [Case Setup](#Case-Setup)
    - [Step 1: Loading the Data](#Step-1:-Loading-the-Data)
    - [Step 2: Creating the nodes](#Step-2:-Creating-the-nodes)
    - [Step 3: Creating the Domain, defining the Constraints and Validators](#Step-3:-Creating-the-Domain,-defining-the-Constraints-and-Validators)
    - [Step 4: Adding the Validator](#Step-4:-Adding-the-Validator)
    - [Step 5: Hydra configuration](#Step-5:-Hydra-configuration)
    - [Step 6: Solver and Training the Model](#Step-6:-Solver-and-Training-the-Model)
    - [Visualizing the solution](#Visualizing-the-solution)

#### Learning Outcomes
- How to use PhysicsNeMo for setting up data-driven problem using Fourier Neural Operator (FNO)
- How to load grid data and setup data-driven constraints
- How to use eager and lazy data loading

## Introduction to Data-driven approach

Let us revisit the following diagram from the introduction notebook. 

<center><img src="images/modulus.webp" alt="Drawing" style="center"/></center>

This diagram below represents the wide range of capabilities that are possessed by Neural networks and while we have explored the Physics Informed approach and Data assimilation approach of them, let us now give a short introduction to the data-driven approach. The data-driven approach involves using large datasets to train models and make predictions or decisions. These large datasets help neural networks learn the features present in unstructured data (images,text etc..); however, it is important to ensure that the data being used is representative of the problem being solved and that the model is evaluated thoroughly to avoid overfitting and other issues.

We will be using the same six-step approach introduced to us earlier for the Data-driven approach, they are as follows:

- **Step 1** : *Geometry and Data*
       
- **Step 2** : *Defining the PDEs and/or creating the nodes*
        
- **Step 3** : *Defining the Constraints* 
   
- **Step 4** : *Creating a domain and adding Constraints, Inferencers, Validators, and Monitors*
    
- **Step 5** : *Setting up the Hydra configuration file*
    
- **Step 6** : *Instantiating the Solver and training the network* 


With a quick introduction to the Data-driven approach and reviewing the six-step approach, let us proceed forward solving Data-driven problems in PhysicsNeMo.

## Theory of the Fourier Neural Operator

Fourier neural operator (FNO) is a data-driven architecture which can be used to parameterize solutions for a distribution of PDE solutions. The key feature of FNO is the spectral convolutions: operations that place the integral kernel in Fourier space. The spectral convolution (Fourier integral operator) is defined as follows:
\begin{equation}
(\mathcal{K}(\mathbf{w})\phi)(x) = \mathcal{F}^{-1}(R_{\mathbf{W}}\cdot \left(\mathcal{F}\right)\phi)(x), \quad \forall x \in D
\end{equation}
where $\mathcal{F}$ and $\mathcal{F}^{-1}$ are the forward and inverse Fourier transforms, respectively.
$R_{\mathbf{w}}$ is the transformation which contains the learnable parameters $\mathbf{w}$. Note this operator is calculated
over the entire <em>structured Euclidean</em> domain $D$ discretized with $n$ points.
Fast Fourier Transform (FFT) is used to perform the Fourier transforms efficiently, and the resulting transformation $R_{\mathbf{w}}$ is just a finite size matrix of learnable weights. In side the spectral convolution, the Fourier coefficients are truncated to only the lower modes which intern allows explicit control over the dimensionality of the spectral space and linear operator.
The FNO model is the composition of a fully-connected "lifting" layer, $L$ spectral convolutions with point-wise linear skip connections and a decoding point-wise fully-connected neural network at the end.
\begin{equation}
u_{net}(\Phi;\theta) = \mathcal{Q}\circ \sigma(W_{L} + \mathcal{K}_{L}) \circ ... \circ \sigma(W_{1} + \mathcal{K}_{1})\circ \mathcal{P}(\Phi), \quad \Phi=\left\{\phi(x); \forall x \in D\right\}
\end{equation}
in which $\sigma(W_{i} + \mathcal{K}_{i})$ is the spectral convolution layer $i$ with the point-wise linear transform $W_{i}$ and activation function $\sigma(\cdot)$. $\mathcal{P}$ is the point-wise lifting network that projects the input into a higher-dimensional latent space, $\mathcal{P}: \mathbb{R}^{d_in} \rightarrow \mathbb{R}^{k}$.
Similarly $\mathcal{Q}$ is the point-wise fully-connected decoding network, $\mathcal{P}: \mathbb{R}^{k} \rightarrow \mathbb{R}^{d_out}$. Since all fully-connected components of FNO are point-wise operations, the model is invariant to the dimensionality of the input.

<strong>Note:</strong> While FNO is technically invariant to the dimensionality of the discretized domain $D$, this domain <em>must</em> be a structured grid in Euclidean space. The inputs to FNO are analogous to images, but the model is invariant to the image resolution.


## Solving the Darcy-Flow problem

### Problem Description 
We will demonstrate the use of Fourier Neural Operators on a 2D Darcy flow problem. The Darcy PDE is a second-order, elliptic PDE with the following form:

\begin{equation}
-\nabla \cdot \left(k(\textbf{x})\nabla u(\textbf{x})\right) = f(\textbf{x}), \quad \textbf{x} \in D,
\end{equation}
in which $u(\textbf{x})$ is the flow pressure, $k(\textbf{x})$ is the permeability field and $f(\cdot)$ is the
forcing function. The Darcy flow can parameterize a variety of systems, including flow through porous media, elastic materials 
and heat conduction. Here you will define the domain as a 2D unit square  $D=\left\{x,y \in (0,1)\right\}$ with the boundary condition $u(\textbf{x})=0, \textbf{x}\in\partial D$. Recall that FNO requires a structured Euclidean input such that $D = \textbf{x}_{i}$ where $i \in \mathbb{N}_{N\times N}$. Thus both the permeability and flow fields are discretized into a 2D matrix $\textbf{K}, \textbf{U} \in \mathbb{R}^{N \times N}$.
This problem develops a surrogate model that learns the mapping between a permeability field and the pressure field,
$\textbf{K} \rightarrow \textbf{U}$, for a distribution of permeability fields $\textbf{K} \sim p(\textbf{K})$.
This is a key distinction of this problem from other PINN examples, you are <em>not</em> learning just a single solution but rather a distribution.
<center><img src="images/fno_darcy.png" alt="Drawing" style="width:900px" /></center>


The example covered in this notebook is a data-driven problem. This means that before starting any coding, we need to make sure that we have both the training and the validation data. The training and validation datasets for this example can be found on the [Fourier Neural Operator Github page](https://github.com/zongyi-li/fourier_neural_operator). The script [`utilities.py`](../../source_code/darcy/utilities.py) is an automated script for downloading and converting this dataset.

<strong>Note:</strong> In this notebook, we will walk through the contents of the <a href="../../source_code/darcy/darcy_FNO_lazy.py" rel="nofollow"><code>darcy_FNO_lazy.py</code></a> script.


### Case Setup 

Now, let's start with importing the required packages and modules

```python
import physicsnemo
from physicsnemo.sym.hydra import to_absolute_path, instantiate_arch, PhysicsNeMoConfig
from physicsnemo.sym.key import Key

from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.domain.constraint import SupervisedGridConstraint
from physicsnemo.sym.domain.validator import GridValidator
from physicsnemo.sym.dataset import HDF5GridDataset

from physicsnemo.sym.utils.io.plotter import GridValidatorPlotter

from utilities import download_FNO_dataset
```

### Step 1: Loading the Data

For this data-driven problem, the first step is to get the training data into PhysicsNeMo. Before loading data, we can set any normalization value that we want to apply to the data. For this dataset, we calculated the scale and shift parameters for both the input permeability field and output pressure. Then, we set this normalization inside PhysicsNeMo by providing the scale/shift to each key, <code>Key(name, scale=(shift, scale))</code>. 


```python
@physicsnemo.sym.main(config_path="conf", config_name="config_FNO")
def run(cfg: PhysicsNeMoConfig) -> None:

    # load training/ test data
    input_keys = [Key("coeff", scale=(7.48360e00, 4.49996e00))]
    output_keys = [Key("sol", scale=(5.74634e-03, 3.88433e-03))]
```

There are two approaches for loading data: First, use eager loading, where you immediately read the entire dataset into memory at one time. Alternatively, you can use lazy loading, where the data is loaded on a per-example basis as the model needs it for training. The eager loading eliminates potential overhead from reading data from disk during training, however, this cannot scale to large datasets. Lazy loading is used in this example for the training dataset to demonstrate this utility for larger problems. This data is in HDF5 format, which is ideal for lazy loading using the *HDF5DataFile* object.


```python
    download_FNO_dataset("Darcy_241", outdir="datasets/")
    train_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5")
    test_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5")

    # make datasets
    train_dataset = HDF5GridDataset(train_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=1000)
    test_dataset = HDF5GridDataset(test_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=100)
```

**Note:** The key difference when setting up eager versus lazy loading is the object passed in the variable dictionaries *invar_train* and *outvar_train*. In eager loading, these dictionaries should be of the type `Dict[str: np.array]`, where each variable is a numpy array of data. Lazy loading uses dictionaries of the type `Dict[str: DataFile]`, consisting of `DataFile` objects which are classes that are used to map between the example index and the data file.  


### Step 2: Creating the nodes

Initializing the model and domain follows the same steps as the other PINN models we saw earlier. 

```python
    # make list of nodes to unroll graph on
    decoder_net = instantiate_arch(
        cfg=cfg.arch.decoder,
        output_keys=output_keys,
    )
    fno = instantiate_arch(
        cfg=cfg.arch.fno,
        input_keys=input_keys,
        decoder_net=decoder_net,
    )
    nodes = [fno.make_node('fno')]

```

### Step 3: Creating the Domain, defining the Constraints and Validators

For the physics-informed problems in PhysicsNeMo, we typically need to define geometry and constraints based on boundary conditions and governing equations. Here, the only constraint is a <code>SupervisedGridConstraint</code>, which performs standard supervised training on grid data. This constraint supports the use of multiple workers, which is particularly important when using lazy loading. 


```python
    # make domain
    domain = Domain()
    
    # add constraints to domain
    supervised = SupervisedGridConstraint(
        nodes=nodes,
        dataset=train_dataset,
        batch_size=cfg.batch_size.grid,
        num_workers=4,  # number of parallel data loaders
    )
    domain.add_constraint(supervised, "supervised")
```

**Note:** Grid data refers to data that can be defined in a tensor like an image. Inside PhysicsNeMo, this grid of data typically represents a spatial domain and should follow the standard dimensionality of `[batch, channel, xdim, ydim, zdim]` where channel is the dimensionality of your state variables. Both Fourier and convolutional models use grid-based data to efficiently learn and predict entire domains in one forward pass, which is in contrast to the pointwise predictions of standard PINN approaches. 

### Step 4: Adding the Validator

The validation data is then added to the domain using `GridValidator` which should be used when dealing with structured data. 

```python
    # add validator
    val = GridValidator(
        nodes,
        dataset=test_dataset,
        batch_size=cfg.batch_size.validation,
        plotter=GridValidatorPlotter(n_examples=5),
    )
    domain.add_validator(val, "test")
```

### Step 5: Hydra configuration 

 Let's have a quick look at the configuration for this problem. The configuration for this problem is fairly standard within PhysicsNeMo. A specific FNO architecture is defined for this example inside of the config file. These settings were derived through an automated hyper-parameter sweep using Hydra multirun. The most important parameter for the FNO models is the <code>dimension</code>, which tells PhysicsNeMo to load a 1D, 2D or 3D FNO architecture. <code>nr_fno_layers</code> are the number of Fourier convolution layers in the model, and <code>fno_layer_size</code> is the size of the latent embedded features inside the model. The contents of the <a href="../../source_code/darcy/conf/config_FNO.yaml" rel="nofollow"><code>config_FNO.yaml</code></a> are shown below. 


```yaml
defaults :
  - physicsnemo_default
  - /arch/conv_fully_connected_cfg@arch.decoder
  - /arch/fno_cfg@arch.fno
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

arch:
  decoder:
    input_keys: [z, 32]
    output_keys: sol
    nr_layers: 1
    layer_size: 32

  fno:
    input_keys: coeff
    dimension: 2
    nr_fno_layers: 4
    fno_modes: 12
    padding: 9

scheduler:
  decay_rate: 0.95
  decay_steps: 1000

training:
  rec_results_freq : 1000
  max_steps : 10000

batch_size:
  grid: 32
  validation: 32
```

### Step 6: Solver and Training the Model 

We can create a solver by using the domain we just created along with the other configurations that define the optimizer choices, and settings using PhysicsNeMo’ `Solver` class. The solver can then be executed using the solve method

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

Before we can start training, we can make use of Tensorboard for visualizing the loss values and convergence of several other monitors we just created. This can be done inside the Jupyter framework by selecting the directory in which the checkpoint will be stored by clicking on the small checkbox next to it. The option to launch a Tensorboard then shows up in that directory. Once you open Tensorboard, switch between the SCALARS , IMAGES , TEXT , TIME SERIES to visualise and view Validation and other information related to Training.


For this application, please verify if you are inside the `/jupyter_notebook/Operators` folder after launching Tensorboard.

1. The option to launch a Tensorboard then shows up in that directory.

<center><img src="../projectile/images/tensorboard.png" alt="Drawing" style="width:900px" /></center>

2. We can launch tensorboard using the following command: 

```
tensorboard --logdir /workspace/python/jupyter_notebook/ --port 8889
```

3. Open a new tab in your browser and head to [http://127.0.0.1:8889](http://127.0.0.1:8889) . You should see a screen similar to the below one. 

<center><img src="../projectile/images/tensorboard_browser.png" alt="Drawing" style="width:900px" /></center>


The training for the problem can be simply started by executing the python script similar to the examples we saw earlier. 

In [ ]:
import os
os.environ["RANK"]="0"
os.environ["WORLD_SIZE"]="1"
os.environ["MASTER_ADDR"]="localhost"

In [ ]:
!python ../../source_code/darcy/darcy_FNO_lazy.py

### Visualizing the solution

The checkpoint directory is saved based on the results recording frequency specified in the `rec_results_freq` parameter of its derivatives. The network directory folder contains several plots of different validation predictions. Several are shown below, and you can see that the model is able to accurately predict the pressure field for permeability fields it had not seen previously. 

FNO validation predictions. (Left to right) Input permeability, true pressure, predicted pressure, error. 

<center><img src="images/fno_darcy_pred1.png" alt="Drawing" style="width: 900px;"/></center>
<center><img src="images/fno_darcy_pred2.png" alt="Drawing" style="width: 900px;"/></center>
<center><img src="images/fno_darcy_pred3.png" alt="Drawing" style="width: 900px;"/></center>


--- 

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---

# Licensing

Copyright © 2023 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.